
# SentimentBayesLSTM: Hybrid Forecasting with GPT Insights and Bayesian Tuning

This project focuses on using LSTM models to predict stock prices based on historical data, sentiment analysis, and technical indicators. The code includes a base `StockPredictor` class, a standard `LSTMModel`, and an `ImprovedLSTMModel` with attention mechanisms and bidirectional LSTMs. Additionally, Bayesian optimization is applied to fine-tune the model's hyperparameters.

## Import Libraries

We will begin by importing the necessary libraries for data processing, model building, and visualization.


In [ ]:
!pip install numpy pandas scikit-learn tensorflow matplotlib requests beautifulsoup4 openai tqdm yfinance scikit-optimize ta


In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional, Attention, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Model
import matplotlib.dates as mdates
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
from openai import OpenAI
from tqdm import tqdm
import time
import yfinance as yf
from skopt import gp_minimize
from skopt.space import Real, Integer
from skopt.utils import use_named_args
from tensorflow.keras import Input
import ta
import os
import json
import tensorflow as tf
print("All libraries imported successfully!")


## 1. Set Up OpenAI API




We will begin by importing the necessary libraries for data processing, model building, and OpenAI API interaction. You'll need to set your OpenAI API key to access GPT functionality. Please replace 'YOUR_OPENAI_API_KEY_HERE' with your actual OpenAI API key from https://platform.openai.com/api-keys

In [ ]:
api_key = 'YOUR_OPENAI_API_KEY_HERE'  # Replace with your actual OpenAI API key
openai_client = OpenAI(api_key=api_key)

def test_openai_client():
    try:
        response = openai_client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": "Hello, are you working?"}
            ]
        )
        print("OpenAI client is working. Response:", response.choices[0].message.content)
    except Exception as e:
        print(f"Error testing OpenAI client: {str(e)}")

test_openai_client()


## 2. SentimentAnalysis Class

This class handles sentiment analysis and is independent of the other classes. However, if we intend to integrate sentiment analysis as a feature in the stock prediction models, it should be defined before the `StockPredictor` class.


In [ ]:
class SentimentAnalyzer:
    def __init__(self, openai_client):
        self.openai_client = openai_client
        self.cache = {}

    def get_google_news_headlines(self, ticker, date):
        url = f"https://www.google.com/search?q={ticker}+stock&tbm=nws&source=lnt&tbs=cdr:1,cd_min:{date},cd_max:{date}"
        headers = {"User-Agent": "Mozilla/5.0"}
        try:
            response = requests.get(url, headers=headers)
            soup = BeautifulSoup(response.text, 'html.parser')
            headlines = soup.find_all('div', class_='BNeawe vvjwJb AP7Wnd')
            return [headline.text for headline in headlines]
        except Exception as e:
            print(f"Error fetching headlines: {e}")
            return []

    def get_openai_sentiment_batch(self, headlines):
        if not headlines:
            return []

        prompt = "Analyze the sentiment of the following headlines and provide a sentiment score from -1 (very negative) to 1 (very positive) for each. Respond with only the numbers, separated by commas:\n\n"
        prompt += "\n".join(headlines)

        try:
            response = self.openai_client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": "You are a financial sentiment analyzer. Always respond with numbers between -1 and 1, separated by commas."},
                    {"role": "user", "content": prompt}
                ],
                max_tokens=100
            )

            content = response.choices[0].message.content.strip()
            sentiment_scores = [float(score.strip()) for score in content.split(',')]
            return [max(min(score, 1), -1) for score in sentiment_scores]
        except Exception as e:
            print(f"Error getting sentiment: {e}")
            return [0] * len(headlines)  # Return neutral sentiment in case of error

    def add_weekly_sentiment(self, df, ticker):
        cache_file = f"{ticker}_sentiment_cache.json"
        df['Week'] = df.index.to_period('W')

        if os.path.exists(cache_file):
            with open(cache_file, 'r') as f:
                self.cache = json.load(f)
            print(f"Loaded cached sentiment data for {ticker}")
        else:
            self.cache = {}

        weeks_to_process = [week for week in df['Week'].unique() if str(week) not in self.cache]
        print(f"Total weeks: {len(df['Week'].unique())}, Weeks to process: {len(weeks_to_process)}")

        for week in tqdm(weeks_to_process, desc="Fetching weekly sentiment"):
            week_str = str(week)
            week_start = week.start_time.strftime('%Y-%m-%d')
            week_end = week.end_time.strftime('%Y-%m-%d')
            headlines = self.get_google_news_headlines(ticker, f"{week_start},{week_end}")
            print(f"Found {len(headlines)} headlines for week {week_str}")

            if headlines:
                sentiments = self.get_openai_sentiment_batch(headlines)
                sentiment = np.mean(sentiments)
                self.cache[week_str] = sentiment
                print(f"Sentiment for week {week_str}: {sentiment}")
            else:
                print(f"No headlines found for week {week_str}, using neutral sentiment")
                self.cache[week_str] = 0

            # Save updated cache after each new sentiment calculation
            with open(cache_file, 'w') as f:
                json.dump(self.cache, f)

            time.sleep(3)  # Increased sleep time to 3 seconds

        df['Sentiment'] = df['Week'].astype(str).map(self.cache)
        df.drop('Week', axis=1, inplace=True)
        return df

print("Updated SentimentAnalyzer class with batch processing and increased sleep time defined successfully!")

## 3. Define the StockPredictor Base Class

This class will handle downloading stock data, adding technical indicators, and preparing data for training.


In [ ]:
class StockPredictor:
    def __init__(self, look_back=60):
        self.look_back = look_back
        self.scaler = MinMaxScaler(feature_range=(0, 1))
        self.feature_names = [
            'Close', 'Sentiment', 'SMA20', 'SMA50', 'EMA20', 'RSI', 'MACD',
            'MACD_Signal', 'Bollinger_High', 'Bollinger_Low', 'ATR', 'OBV',
            'KAMA', 'Stochastic', 'Williams_R'
        ]

    def download_stock_data(self, ticker, start_date, end_date):
        # Download historical stock data using yfinance.
        stock = yf.Ticker(ticker)
        df = stock.history(start=start_date, end=end_date)
        return df

    def add_technical_indicators(self, df):
        # Calculate and add advanced technical indicators to the dataframe.
        df['SMA20'] = ta.trend.sma_indicator(df['Close'], window=20)
        df['SMA50'] = ta.trend.sma_indicator(df['Close'], window=50)
        df['EMA20'] = ta.trend.ema_indicator(df['Close'], window=20)
        df['RSI'] = ta.momentum.rsi(df['Close'], window=14)
        macd = ta.trend.MACD(df['Close'])
        df['MACD'] = macd.macd()
        df['MACD_Signal'] = macd.macd_signal()
        bollinger = ta.volatility.BollingerBands(df['Close'])
        df['Bollinger_High'] = bollinger.bollinger_hband()
        df['Bollinger_Low'] = bollinger.bollinger_lband()
        df['ATR'] = ta.volatility.average_true_range(df['High'], df['Low'], df['Close'])
        df['OBV'] = ta.volume.on_balance_volume(df['Close'], df['Volume'])

        # Advanced indicators
        df['KAMA'] = ta.momentum.kama(df['Close'], window=10)
        df['Stochastic'] = ta.momentum.stoch(df['High'], df['Low'], df['Close'])
        df['Williams_R'] = ta.momentum.williams_r(df['High'], df['Low'], df['Close'])

        # Handle NaN values after adding indicators
        df.ffill(inplace=True)  # Forward fill to handle NaNs
        df.dropna(inplace=True)  # Drop any remaining NaNs
        return df

    def prepare_data(self, scaled_data):
        x, y = [], []
        for i in range(len(scaled_data) - self.look_back):
            x.append(scaled_data[i:i+self.look_back])
            y.append(scaled_data[i+self.look_back, 0])  # Assume 'Close' is the first column
        return np.array(x), np.array(y).reshape(-1, 1)

print("StockPredictor class defined successfully!")

## 3. Define the `LSTMModel` Class with Hyperparameter Tuning and Uncertainty Estimation

This version of the `LSTMModel` class extends `StockPredictor` and introduces several key features:
- **Hyperparameter Tuning**: Bayesian optimization via `gp_minimize` to find the best values for LSTM units, dropout rate, and learning rate.
- **Training**: The model is trained twice, first during hyperparameter tuning and then with the best parameters found.
- **Uncertainty Estimation**: The `predict_with_uncertainty` method performs multiple forward passes with dropout enabled to estimate prediction uncertainty.


In [ ]:
class LSTMModel(StockPredictor):
    def __init__(self, look_back=60, forecast_horizon=1):
        super().__init__(look_back)
        self.forecast_horizon = forecast_horizon
        self.model = self.build_model()

    def build_model(self, lstm_units=128, dropout_rate=0.2, learning_rate=0.001):
        lstm_units = int(lstm_units)
        input_shape = (self.look_back, len(self.feature_names))

        inputs = Input(shape=input_shape)
        x = LSTM(units=lstm_units, return_sequences=True)(inputs)
        x = Dropout(dropout_rate)(x)
        x = LSTM(units=lstm_units, return_sequences=True)(x)
        x = Dropout(dropout_rate)(x)
        x = LSTM(units=lstm_units)(x)
        x = Dropout(dropout_rate)(x)
        outputs = Dense(units=self.forecast_horizon)(x)

        model = Model(inputs=inputs, outputs=outputs)
        model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mean_squared_error')
        print("Model Summary:")
        model.summary()
        print(f"Input shape: {input_shape}")
        return model

    def train(self, x_train, y_train):
        print("In train method:")
        print(f"x_train shape: {x_train.shape}")
        print(f"y_train shape: {y_train.shape}")
        print(f"look_back: {self.look_back}")
        print(f"Number of features: {len(self.feature_names)}")

        space = [
            Integer(64, 256, name='lstm_units'),
            Real(0.1, 0.5, name='dropout_rate'),
            Real(1e-4, 1e-2, name='learning_rate', prior='log-uniform')
        ]

        @use_named_args(space)
        def objective(**params):
            model = self.build_model(**params)
            try:
                history = model.fit(
                    x_train, y_train,
                    epochs=50,
                    batch_size=32,
                    validation_split=0.2,
                    verbose=0
                )
                return -history.history['val_loss'][-1]
            except Exception as e:
                print(f"Error during model fitting: {str(e)}")
                return np.inf

        result = gp_minimize(objective, space, n_calls=50, random_state=42)

        best_params = {
            'lstm_units': int(result.x[0]),
            'dropout_rate': result.x[1],
            'learning_rate': result.x[2]
        }
        print("Best parameters found: ", best_params)
        print("Best score found: ", -result.fun)

        self.model = self.build_model(**best_params)
        self.model.fit(x_train, y_train, epochs=200, batch_size=32, verbose=1)
    def predict(self, x_test):
            """
            Make predictions using the trained LSTM model.
            :param x_test: Test data
            :return: Predictions from the model
            """
            if self.model is None:
                raise ValueError("Model has not been trained yet.")

            return self.model.predict(x_test)
    def predict_with_uncertainty(self, x, n_iter=100):
        if self.model is None:
            raise ValueError("Model has not been trained. Call train() before making predictions.")

        predictions = []
        for _ in range(n_iter):
            preds = self.model.predict(x, batch_size=32, verbose=0)
            predictions.append(preds)
        predictions = np.array(predictions)

        mean_prediction = predictions.mean(axis=0)
        uncertainty = predictions.std(axis=0)

        mean_prediction_reshaped = mean_prediction.reshape(-1, 1)
        mean_prediction_scaled = self.scaler.inverse_transform(
            np.hstack([mean_prediction_reshaped, np.zeros((mean_prediction_reshaped.shape[0], len(self.feature_names)-1))])
        )[:, 0].reshape(mean_prediction.shape)

        uncertainty_scaled = uncertainty * self.scaler.scale_[0]

        return mean_prediction_scaled, uncertainty_scaled
original_lstm_model = LSTMModel()

## 4. Downloading Stock Data

The first step in building our stock prediction model is to download historical stock data. In this section, we will:
1. Define the parameters for the stock data we want to download (ticker symbol, start date, and end date).
2. Create an instance of the `StockPredictor` class, which encapsulates the logic for downloading and processing stock data.
3. Use the `download_stock_data()` method to fetch historical stock prices for the specified ticker and date range.
4. Display the first few rows of the downloaded data to ensure everything was retrieved correctly.

### Code Explanation:

- **Parameters**: We define the `ticker`, `start_date`, and `end_date` to specify which stock and what date range to download.
- **Instance Creation**: We create an instance of the `StockPredictor` class, which will handle downloading the stock data.
- **Download Stock Data**: The `download_stock_data()` method fetches the data from Yahoo Finance for the given parameters.
- **Display Data**: Finally, we print the first few rows of the data to inspect it and verify that it was downloaded correctly.


In [ ]:
# Define parameters
ticker = "aapl"
start_date = '2020-01-01'
end_date = '2023-12-31'

# Create an instance of StockPredictor
predictor = StockPredictor()

# Download stock data
df = predictor.download_stock_data(ticker, start_date, end_date)
print("Stock data downloaded")

# Display the first few rows of the downloaded data
print(df.head())


## 5. Data Preparation: Splitting, Scaling, and Creating Sequences

In this section, we prepare the data for training our LSTM model by performing the following steps:

1. **Add Technical Indicators**: We enhance the dataset by adding various technical indicators, such as moving averages, RSI, and MACD.
2. **Perform Sentiment Analysis**: We integrate sentiment data into the dataset using the `SentimentAnalyzer`.
3. **Train-Test Split**: We split the dataset into training and testing sets, with 80% of the data used for training and 20% for testing.
4. **Data Scaling**: We scale the features using `MinMaxScaler` to ensure that the data is normalized, which helps the LSTM model converge during training.
5. **Creating Sequences for LSTM**: We convert the data into sequences of time steps that will be used as input for the LSTM model.
6. **Inspecting Data Shapes**: We output the shapes of the training and testing data to verify that the data preparation process is correct.

### Code Explanation:

- **Technical Indicators**: We use the `StockPredictor` class to add technical indicators to the DataFrame.
- **Sentiment Analysis**: Sentiment data is added to the DataFrame on a weekly basis using the `SentimentAnalyzer`.
- **Train-Test Split**: The dataset is split into training and testing sets using an 80/20 split.
- **Data Scaling**: We scale the relevant features using `MinMaxScaler`.
- **Creating Sequences**: The data is transformed into sequences suitable for training an LSTM model, which requires sequential data as input.
- **Inspecting Data Shapes**: We print the shapes of the training and testing datasets to ensure that they are correctly prepared for model training.


In [ ]:
# Ensure predictor is initialized (if not done already)
predictor = StockPredictor(look_back=60)  # Adjust look_back as needed

# Add technical indicators
print("Adding technical indicators...")
df = predictor.add_technical_indicators(df)
print("Technical indicators added")

# Perform sentiment analysis
sentiment_analyzer = SentimentAnalyzer(openai_client)
print("Starting sentiment analysis...")
df = sentiment_analyzer.add_weekly_sentiment(df, ticker)
print("Sentiment analysis completed")

# Scale the entire dataset
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df[predictor.feature_names].values)

# Split the data into training and test sets
train_size = int(len(scaled_data) * 0.8)

# Prepare LSTM sequences
print("Preparing data for LSTM...")
x_train, y_train = predictor.prepare_data(scaled_data[:train_size])
x_test, y_test = predictor.prepare_data(scaled_data[train_size:])
print("Data preparation completed")

# Print data shapes
print(f"x_train shape: {x_train.shape}, y_train shape: {y_train.shape}")
print(f"x_test shape: {x_test.shape}, y_test shape: {y_test.shape}")

# Verify that test set is not empty
if x_test.size == 0 or y_test.size == 0:
    raise ValueError("Test set is empty. Please check your data splitting and preparation steps.")

## 6. Model Initialization, Training, and Evaluation with Bayesian Optimization and Error Handling

In this section, we focus on initializing, training, and evaluating the LSTM model, with added support for Bayesian optimization to find the optimal hyperparameters. We also include robust error handling to ensure the smooth execution of the training and evaluation processes.

### Steps Involved:

1. **Model Initialization**: We initialize the LSTM model by defining its architecture, which includes multiple LSTM layers and dropout for regularization.
2. **Bayesian Optimization for Hyperparameter Tuning**: We perform Bayesian optimization to automatically find the best hyperparameters for the LSTM model, such as the number of LSTM units, dropout rate, and learning rate.
3. **Model Training**: After finding the optimal hyperparameters, we retrain the LSTM model on the training data using these optimized settings.
4. **Model Evaluation and Prediction**: Once training is complete, we make predictions on the test data and calculate performance metrics such as Mean Squared Error (MSE) and Mean Absolute Error (MAE). Additionally, we can estimate prediction uncertainty using Monte Carlo Dropout.
5. **Error Handling**: A `try-except` block is used throughout the training and optimization process to catch and handle any errors that may arise, ensuring the script continues execution without crashing.

### Code Explanation:

- **Model Initialization**: The LSTM model is initialized using the `LSTMModel` class, which includes multiple LSTM layers with dropout for regularization. The model architecture can be customized through hyperparameters.
  
- **Bayesian Optimization**: The `train()` method leverages Bayesian optimization (`gp_minimize`) to search for the optimal hyperparameters. The optimization process minimizes the validation loss by testing different configurations of LSTM units, dropout rates, and learning rates.

- **Model Training**: After finding the optimal hyperparameters, the model is retrained on the full training dataset using these settings for 200 epochs.

- **Model Evaluation and Prediction**: Predictions are made on the test data, and we calculate metrics such as MSE and MAE to evaluate performance. Additionally, `predict_with_uncertainty()` can be used to provide uncertainty estimates for the predictions, which can be useful for understanding the model's confidence.

- **Error Handling**: The `try-except` blocks catch any errors during the model training or evaluation process. If an error occurs during Bayesian optimization or training, it is handled gracefully, and a relevant error message is printed. This prevents the script from crashing and allows the optimization process to continue.


In [ ]:
try:
    # Step 1: Initialize the LSTM model
    original_lstm_model = LSTMModel()

    # Step 2: Data Preparation - Scale the dataset using MinMaxScaler
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(df[original_lstm_model.feature_names].values)

    # Split the data into training and testing sets (80% training, 20% testing)
    train_size = int(len(scaled_data) * 0.8)
    train_data = scaled_data[:train_size]
    test_data = scaled_data[train_size:]

    # Prepare the data for LSTM (create sequences for input)
    x_train, y_train = original_lstm_model.prepare_data(train_data)
    x_test, y_test = original_lstm_model.prepare_data(test_data)

    # Reshape the target data (y_train, y_test) for the model's output format
    y_train = y_train.reshape(-1, 1)  # Reshape to (batch_size, 1)
    y_test = y_test.reshape(-1, 1)    # Reshape to (batch_size, 1)

    # Step 3: Bayesian Optimization for Hyperparameter Tuning

    original_lstm_model.train(x_train, y_train)

except Exception as e:
    # Step 5: Error Handling - Catch any errors in data preparation and print an error message
    print(f"An error occurred during data preparation: {str(e)}")


## 7. Debugging and Validating Input Data for LSTM Model

Before proceeding with model training, it's crucial to validate and debug the input data to ensure that it is properly formatted and free of common issues such as `NaN` values, incorrect shapes, or empty datasets. These issues can cause the model to fail during training or produce unreliable predictions.

In this section, we will:

1. **Define the `debug_input_data` Function**: This function inspects the input data for any potential issues. It checks for `NaN` values, incorrect data shapes, and empty datasets. Where possible, the function will fix minor issues (e.g., replacing `NaN` values) and raise errors for more significant problems.
2. **Apply the Function**: We will apply the `debug_input_data` function to the training and testing datasets before feeding them into the LSTM model. This ensures that the data is in the correct format and meets the necessary requirements for training.
3. **Handle Exceptions**: We use a `try-except` block to catch any exceptions that may arise during data debugging and preparation. If an issue is detected, an appropriate error message will be displayed, and the script will not proceed until the issue is resolved.

### Code Explanation:

- **Defining `debug_input_data`**: The function checks for `NaN` values and replaces them with zeroes, verifies that the dataset is not empty, and ensures that the input data for the LSTM model has the correct shape (3D: `[samples, time steps, features]`).
- **Applying the Function**: We apply the `debug_input_data` function to `x_train`, `y_train`, `x_test`, and `y_test` to ensure that the data is suitable for model training.
- **Handling Errors**: If any issues are detected during data validation, an error message is printed, and the process is halted. This helps prevent the model from training on bad data.


In [ ]:
# Define the utility function to debug and validate input data
def debug_input_data(x_train, y_train, x_test, y_test):
    """
    This function inspects and debugs the input data for common issues such as NaN values,
    incorrect shapes, and empty datasets. It will try to fix minor issues and raise errors for major problems.

    Args:
        x_train (np.array): Training data features
        y_train (np.array): Training data labels
        x_test (np.array): Testing data features
        y_test (np.array): Testing data labels

    Returns:
        tuple: Returns the (x_train, y_train, x_test, y_test) after debugging.
    """
    # Check for NaN values in the data
    if np.isnan(x_train).any() or np.isnan(y_train).any():
        print("Warning: NaN values found in the training data. Replacing NaNs with 0.")
        x_train = np.nan_to_num(x_train)
        y_train = np.nan_to_num(y_train)

    if np.isnan(x_test).any() or np.isnan(y_test).any():
        print("Warning: NaN values found in the test data. Replacing NaNs with 0.")
        x_test = np.nan_to_num(x_test)
        y_test = np.nan_to_num(y_test)

    # Check if the datasets are empty
    if x_train.size == 0 or y_train.size == 0:
        raise ValueError("Training set is empty. Please check your data preparation steps.")

    if x_test.size == 0 or y_test.size == 0:
        raise ValueError("Test set is empty. Please check your data preparation steps.")

    # Ensure that the LSTM input data has the correct shape (3D array: [samples, time steps, features])
    if len(x_train.shape) != 3:
        raise ValueError(f"Incorrect shape for x_train: {x_train.shape}. Expected a 3D array.")

    if len(x_test.shape) != 3:
        raise ValueError(f"Incorrect shape for x_test: {x_test.shape}. Expected a 3D array.")

    print("Input data successfully debugged and verified.")

    return x_train, y_train, x_test, y_test

# Apply the debug_input_data function and handle any exceptions
try:
    # Debug input data to check for issues
    x_train, y_train, x_test, y_test = debug_input_data(x_train, y_train, x_test, y_test)

    # Ensure that the LSTM input has the correct shape
    print("Input shape for LSTM:")
    print(f"x_train shape: {x_train.shape}")
    print(f"x_test shape: {x_test.shape}")

    # Check if the test set is empty
    if x_test.size == 0:
        print("Test set is still empty. Try adjusting the split ratio or inspecting the data preparation logic.")
except Exception as e:
    print(f"An error occurred during data inspection: {str(e)}")

## 8. Making Predictions and Evaluating Model Performance

After training the original LSTM model, the next step is to make predictions on the test data and evaluate the model's performance. In this section, we will:

1. **Make Predictions**: Use the trained LSTM model to make predictions on the test dataset (`x_test`).
2. **Inverse Transform Predictions and Actual Values**: Since the data was scaled during preprocessing, we need to inverse transform the predictions and actual values to return them to their original scale.
3. **Calculate Performance Metrics**: Evaluate the model's performance using metrics such as Mean Squared Error (MSE) and Mean Absolute Error (MAE). These metrics provide insight into the accuracy of the model's predictions.

### Code Explanation:

- **Making Predictions**: The `predict` method of the trained LSTM model is called to generate predictions on the test data.
- **Inverse Transforming Data**: Both the predictions and the actual values (`y_test`) are inverse transformed from the scaled values back to their original scale using the `scaler`. This step is crucial for accurately calculating performance metrics.
- **Calculating Performance Metrics**: We compute the MSE and MAE to quantify the model's prediction accuracy. MSE gives us a sense of the average squared difference between predictions and actual values, while MAE measures the average absolute difference.

In [ ]:
# Make predictions on the test set using the trained LSTM model
predictions = original_lstm_model.predict(x_test)

# Inverse transform the predictions back to their original scale
# We add zeros for the other features that were scaled but not predicted, to maintain the correct shape for inverse transformation
predictions_scaled = np.hstack([predictions, np.zeros((len(predictions), len(original_lstm_model.feature_names)-1))])
predictions_unscaled = scaler.inverse_transform(predictions_scaled)[:, 0]

# Inverse transform the actual test values (y_test) back to their original scale
# Similar to the predictions, we add zeros to maintain the correct shape for inverse transformation
y_test_scaled = np.hstack([y_test, np.zeros((len(y_test), len(original_lstm_model.feature_names)-1))])
y_test_unscaled = scaler.inverse_transform(y_test_scaled)[:, 0]

# Calculate performance metrics to evaluate the model's accuracy
mse = mean_squared_error(y_test_unscaled, predictions_unscaled)
mae = mean_absolute_error(y_test_unscaled, predictions_unscaled)

# Print the calculated performance metrics to assess the model's performance
print(f"Mean Squared Error: {mse}")
print(f"Mean Absolute Error: {mae}")

## 9. Inspecting Shapes and Model Features

After making predictions and calculating performance metrics, it's important to inspect the shapes of the actual and predicted values, as well as the number of features used by the model. Ensuring that the shapes and feature counts are consistent is crucial for debugging and verifying that the model is processing data as expected.

### Steps Involved:

1. **Inspecting Shapes of `y_test` and Predictions**: We print the shapes of the actual test values (`y_test`) and the predicted values to verify that they match. Consistent shapes are necessary for calculating performance metrics and ensuring that the model is functioning correctly.
2. **Inspecting Number of Features**: We print the number of features used by the model. This helps confirm that the model is processing the correct number of input features, which is essential for ensuring that all relevant data is being considered during training and prediction.

### Code Explanation:

- **Inspecting Shapes**: We print the shapes of `y_test` and `predictions` to check if they match. Any discrepancies in these shapes would indicate an issue in the data preparation or model prediction steps.
- **Inspecting Features**: We print the number of features used by the model, which helps verify that the correct features were included during training and prediction.


In [ ]:
# Inspect and print the shape of the actual test values (y_test) to verify its dimensions
print("Shape of y_test:", y_test.shape)

# Inspect and print the shape of the predicted values to ensure it matches the shape of y_test
print("Shape of predictions:", predictions.shape)

# Print the number of features used by the original LSTM model during training and prediction
print("Number of features:", len(original_lstm_model.feature_names))


## 10. Rescaling Predictions and Plotting Stock Prices

In this section, we rescale the predicted and actual stock prices back to their original range and visualize the results using a line plot. Rescaling the data is necessary because the data was previously scaled during preprocessing, and we need to convert it back to the original scale for meaningful interpretation and comparison. We also print relevant statistics, such as the date range and price range.

### Steps Involved:

1. **Get Original Price Range**: Extract the minimum and maximum values of the stock prices from the training data (`df['Close']`). These values are needed to rescale the data back to its original range.
2. **Define Rescaling Function**: Create a function that converts the scaled data back to the original price range using the previously extracted minimum and maximum values.
3. **Rescale Data**: Apply the rescaling function to both the actual test values (`y_test`) and the predicted values (`predictions`) to convert them back to the original scale.
4. **Plot the Results**: Plot the rescaled actual and predicted prices against the test dates to visualize the model's performance. We customize the plot with labels, legends, and an improved x-axis format for better readability.
5. **Print Statistics**: Display relevant statistics such as the date range, adjusted price range, and the original data's price range.

### Code Explanation:

- **Rescaling Function**: We define a function, `rescale_to_original`, that takes scaled data and rescales it to the original price range using the minimum and maximum prices.
- **Plotting**: The actual and predicted prices are plotted on the same chart to visually compare the model's performance. The plot is customized with a title, labels, and a grid, and the x-axis date formatting is improved for better readability.
- **Printing Statistics**: We print the date range, adjusted price range (after rescaling), and the original price range to provide additional context about the data being plotted.

In [ ]:
# Get the original price range from your training data
original_min_price = df['Close'].min()
original_max_price = df['Close'].max()

# Function to rescale the data back to original price range
def rescale_to_original(scaled_data, original_min, original_max):
    return scaled_data * (original_max - original_min) + original_min

# Rescale y_test and predictions
y_test_unscaled = rescale_to_original(y_test, original_min_price, original_max_price)
predictions_unscaled = rescale_to_original(predictions, original_min_price, original_max_price)

# Create test_dates (adjust the start date as needed)
start_date = datetime(2023, 1, 1)  # Replace with your actual start date
test_dates = [start_date + timedelta(days=i) for i in range(len(y_test))]

# Now plot with the corrected data
plt.figure(figsize=(15, 7))
plt.plot(test_dates, y_test_unscaled, label='Actual Price', color='blue')
plt.plot(test_dates, predictions_unscaled, label='Predicted Price', color='red')

plt.title(f'{ticker} Stock Price Prediction')
plt.xlabel('Date')
plt.ylabel('Price ($)')
plt.legend()

# Improve x-axis date formatting
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
plt.gca().xaxis.set_major_locator(mdates.MonthLocator(interval=1))
plt.gcf().autofmt_xdate()

max_price = max(np.max(y_test_unscaled), np.max(predictions_unscaled))
min_price = min(np.min(y_test_unscaled), np.min(predictions_unscaled))
plt.ylim(min_price * 0.9, max_price * 1.1)  # Add 10% padding on both ends

plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

# Print statistics
print(f"Date range: {test_dates[0].strftime('%Y-%m-%d')} to {test_dates[-1].strftime('%Y-%m-%d')}")
print(f"Adjusted price range: ${min_price:.2f} to ${max_price:.2f}")
print(f"Original data adjusted price range: ${original_min_price:.2f} to ${original_max_price:.2f}")

## 11. Inspecting Data Structure and Samples

In this final section, we inspect the structure and data types of the test set (`y_test`) and the model's predictions. Understanding the structure of your data is crucial for debugging and ensuring that the data is in the expected format. Additionally, printing a sample of the data allows for a quick verification that everything is working as expected.

### Steps Involved:

1. **Inspecting the Number of Features**: We print the number of features used by the model to ensure that the correct features were included during training and prediction.
2. **Inspecting Shapes**: We check the shapes of `y_test` and `predictions` to verify that they match, which is necessary for accurate performance evaluation.
3. **Inspecting Data Samples**: Printing the first few elements of both `y_test` and `predictions` allows for a quick visual inspection of the data and model output, helping to catch any obvious errors.
4. **Inspecting Data Types**: We check the data types of `y_test` and `predictions` to confirm that they are the expected types (e.g., `numpy.ndarray`). Ensuring the correct data type is important for compatibility with various operations and functions.

### Code Explanation:

- **Number of Features**: The number of features used by the model is printed to verify that the model is working with the correct input data.
- **Shapes**: We print the shapes of `y_test` and `predictions` to ensure that they are consistent and that there are no mismatches.
- **Sample Data**: Printing the first five elements of `y_test` and `predictions` gives a quick overview of what the actual and predicted values look like, allowing for basic validation.
- **Data Types**: The data types of `y_test` and `predictions` are printed to ensure they are compatible with further operations in the workflow.


In [ ]:
print("Number of features:", len(original_lstm_model.feature_names))
print("Shape of y_test:", y_test.shape)
print("Shape of predictions:", predictions.shape)
print("Sample of y_test (first 5 elements):", y_test[:5])
print("Sample of predictions (first 5 elements):", predictions[:5])
print("Type of y_test:", type(y_test))
print("Type of predictions:", type(predictions))